In [37]:
from pathlib import Path
import glob
import pandas as pd
from lib.utils import config

In [67]:
# Read all csv files in the data folder and subfolders
DataDir = Path(config['Dir']['DataDir'])
data_files = glob.glob(str(DataDir/'**'/'IVg*.csv'), recursive=True)
data = [pd.read_csv(f, comment='#') for f in data_files]
if not data: raise ValueError('No data files found in ' + DataDir)

In [154]:
def to_float(val: str) -> str|float:
    """Transforms a string containing data in SI units, or just a number,
    and transforms it to float if possible.
    """
    if val.endswith((' s', ' V', ' A')):
        val = val[:-2]
    try:
        return float(val)
    except ValueError:
        return val

def get_params(file: str):
    """Get the parameters from the file header. Return a dictionary with the
    parameters.
    """
    dic = dict()
    with open(file) as f:
        for line in f:
            if not line.startswith('#'):
                break

            key, value = line.strip('#').strip().split(':')
            value = value.strip()

            if key == 'Procedure':
                # extract 'procedure' from '<__name__.procedure>'
                procedure = value.split('.')[-1][:-1]
                dic['Procedure'] = procedure
                dic['Index'] = file.split(procedure)[-1][:-4]
                dic['Date'] = file.split(procedure)[-1][:10]

            elif not key in ['Parameters', 'Data']:
                dic[key] = to_float(value)

    return dic

In [155]:
param_df = pd.DataFrame([get_params(f) for f in data_files])
param_df.set_index(['Procedure', 'Index'], inplace=True)
param_df.sort_index(inplace=True)

param_df[param_df['Date'] == '2023-05-04']

Date  Irange  N_avg     Chip   Sample  \
Procedure Index                                                       
IVg       2023-05-12_1  2023-05-12   0.001    2.0  Unknown  Unknown   
          2023-05-12_2  2023-05-12   0.001    2.0  Unknown  Unknown   
          2023-05-12_3  2023-05-12   0.001    2.0  Unknown  Unknown   
          2023-05-12_4  2023-05-12   0.001    2.0  Unknown  Unknown   
          2023-05-12_5  2023-05-12   0.001    2.0  Unknown  Unknown   
          2023-05-12_6  2023-05-12   0.001    2.0  Unknown  Unknown   

                        Step time    VDS  VG end  VG start  VG step Comment  
Procedure Index                                                              
IVg       2023-05-12_1       0.01  0.075    35.0     -35.0      0.2       -  
          2023-05-12_2       0.01  0.075    35.0     -35.0      0.2       -  
          2023-05-12_3       0.01  0.075    35.0     -35.0      0.2       -  
          2023-05-12_4       0.01  0.075    35.0     -35.0      0.2       -  
          2023-05-12_5       0.01  0.075    35.0     -35.0      0.2       -  
          2023-05-12_6       0.01  0.075    35.0     -35.0      0.2       -